In [1]:
#数据处理页面
import pandas as pd
import numpy as np
import math
import re

## 详情页内容处理

In [2]:
#csv内容为空判断
def is_nan(a):
    if type(a)==type(0.1):
        return 1
    else:
        return 0

In [3]:
data = "data/car_detail.csv"
pd_table = pd.read_csv(data, usecols=[1,2,3,4,5,6], encoding='gbk', header=None) # 没有表头
array_table = np.array(pd_table)

print(array_table[1])
print(len(array_table))

['长安-悦翔V3 2012款 1.3L 手动舒适型 ' '1.00万' '新车含税4.89万'
 '7.94万公里行驶里程/7年2个月上牌/青岛车辆所在地/国五外迁查询/手动变速箱/1次 过户记录/3天内上架时间' '急售'
 '该车外观未发现色差， 但局部钣金、局部喷漆；外观有少量瑕疵，外观无更换；灯光系统正常；内饰整洁；电子系统正常；发动机、变速箱工况正常，怠速规律无抖动，转向灵活；综合车况优秀。无重大事故无泡水事故无火烧事故重大事故排查水泡车排查12项火烧车排查5项重大撞击排查17项轻微碰撞排查21项易损易耗部件23项功能部件检测内部配置15项动力、传动系统9项安全系统5项灯光、空调系统8项外观内饰检测内饰检测24项外观检测47项2项启动检测4项']
52749


In [4]:
new_table = []

for i in range(1, 52747, 1):
    
    array_i = []
    
    #print(array_table[i])
    pattern_int = re.compile(r'\d+')
    pattern_float = re.compile(r"\d+\.?\d*")
    #print()
    #-----------------------------------------------------------------------


    #-----------------------------------------------------------------------
    #品牌因素，有研究得知，一般主流品牌更能卖出高价格
    #以人人车网主页显示品牌为主要品牌
    brand = ['大众','丰田','宝马','本田','奔驰','奥迪','别克','日产','福特','现代','雪佛兰','吉利汽车','马自达','比亚迪','长安','起亚','凯迪拉克','标致']
    name = array_table[i][0]
    name = name[0:name.find("-")]
    
    name_num = 0
    for p in brand:
        if name == p:
            name_num = 1
            break
    #print("汽车品牌：", name)
    #-----------------------------------------------------------------------
    
    
    #-----------------------------------------------------------------------
    old_money = array_table[i][1]
    old_money = float(old_money[0:old_money.find("万")])
    array_i.append(old_money)
    #print("二手车价格：", old_money) 
    array_i.append(name_num)#品牌
    #-----------------------------------------------------------------------
    
    
    #-----------------------------------------------------------------------
    new_money = array_table[i][2]
    new_money = float(new_money[new_money.find("税")+1:new_money.find("万")])
    array_i.append(new_money)
    #print("新车购入价格：", new_money) 
    #-----------------------------------------------------------------------


    #-----------------------------------------------------------------------
    summary = array_table[i][3]

    driven_distance = summary[0:summary.find("/")]
    summary = summary[summary.find("/")+1:-1]
    driven_distance = float(driven_distance[0:driven_distance.find("万")])
    array_i.append(driven_distance)
    #print("行驶里程：", driven_distance)
    #-----------------------------------------------------------------------
    
    
    #-----------------------------------------------------------------------
    #使用时间按月计数
    usage_time = summary[0:summary.find("/")]
    summary = summary[summary.find("/")+1:-1]
    usage_time = pattern_int.findall(usage_time)
    if len(usage_time)>1: 
        usage_time = int(usage_time[0])*12 + int(usage_time[1])
    else:
        usage_time = int(usage_time[0])*12
    array_i.append(usage_time)
    #print("使用时间：", usage_time)
    #-----------------------------------------------------------------------
    
    
    #-----------------------------------------------------------------------
    #车牌所在地
    location = summary[0:summary.find("/")]
    summary = summary[summary.find("/")+1:-1]
    location = location[0:location.find("车")]
    
    #根据一线、新一线、其他划分城市（反映上牌难度）
    #一线：
    one = ['北京','上海','广州','深圳']
    #新一线：
    new_one = ['成都','杭州','重庆','武汉','西安','苏州','天津','南京','长沙','郑州','东莞','青岛','沈阳','宁波','佛山','合肥']
    
    location_num = 0
    for p in one:
        if location == p:
            location_num = 2
            break
    
    for p in new_one:
        if location == p:
            location_num = 1
            break
    
    array_i.append(location_num)
    #print("车牌所在地：", location)
    #-----------------------------------------------------------------------
    
    
    #-----------------------------------------------------------------------
    #外迁查询：国三[1,1]上路收到影响，不能外迁。国四[0,1]外迁受到影响，国五[0,0]
    standard = [0,0]
    relocation_standard = summary[0:summary.find("/")]
    summary = summary[summary.find("/")+1:-1]
    if(relocation_standard.find("三") != -1):
        standard = [1,1]
    elif(relocation_standard.find("四") != -1): 
        standard = [0,1]
    elif(relocation_standard.find("五") != -1): 
        standard = [0,0]
    array_i.extend(standard)
    #print("外迁标准：", standard)
    #-----------------------------------------------------------------------
    
    
    #-----------------------------------------------------------------------
    #变速箱：自动1/手动0
    gearbox_result = 0
    gearbox = summary[0:summary.find("/")]
    summary = summary[summary.find("/")+1:-1]
    if(gearbox.find("自动") != -1):
        gearbox_result = 1
    array_i.append(gearbox_result)    
    #print("变速箱：", gearbox_result)
    #-----------------------------------------------------------------------
    
    
    #-----------------------------------------------------------------------
    #过户次数/上架时间(未标注为1天以内的)：
    transfers_num = summary
    a = pattern_int.findall(transfers_num)
    transfers_num = int(a[0])
    array_i.append(transfers_num)    
    #print("过户次数：", transfers_num)

    if len(a)>1:
        added_time = a[1]
    else:
        added_time = 1
    array_i.append(int(added_time))
    #print("上架时间：", added_time)
    #-----------------------------------------------------------------------


    #-----------------------------------------------------------------------
    #特殊标记：
    #超值/急售/0过户/准新车/
    tag_result = [0,0,0,0]

    tags = array_table[i][4]
    if is_nan(tags):
        tag_result = [0,0,0,0]
    else:
        if tags.find("超值") != -1:
            tag_result[0] = 1
        if tags.find("急售") != -1:
            tag_result[1] = 1
        if tags.find("0过户") != -1:
            tag_result[2] = 1
        if tags.find("准新车") != -1:
            tag_result[3] = 1
    array_i.extend(tag_result)
    #print("特殊标记：", tag_result)    
    #-----------------------------------------------------------------------


    #-----------------------------------------------------------------------
    #车辆检测报告详细内容解析
    #综合车况判断，外观问题数目
    #车况：优秀、良好、较好、无描述
    chinese_description = array_table[i][5]
    #print(chinese_description)

    vehicle_condition = 0
    if chinese_description.find("优秀") != -1:
        vehicle_condition = 3
    elif chinese_description.find("良好") != -1:
        vehicle_condition = 2
    elif chinese_description.find("较好") != -1:
        vehicle_condition = 1
    array_i.append(vehicle_condition)
    #print("综合车况：", vehicle_condition)
    #-----------------------------------------------------------------------
    
    
    #-----------------------------------------------------------------------
    external_str = chinese_description[chinese_description.find("外观检测"):chinese_description.find("启动检测")]
    external_num = pattern_int.findall(external_str)
    if len(external_num)>1:
        external_assessment = external_num[1]
    else:
        external_assessment = 0
    array_i.append(int(external_assessment))
    #print("外观检测：", external_assessment)
    #-----------------------------------------------------------------------
    
    if old_money<1 or new_money>100 or vehicle_condition == 0:
        continue
        
    new_table.append(array_i)    

In [5]:
print("汽车品牌/二手车价格/新车购入价格/行驶里程/使用时间/车牌所在地/外迁标准[0,0]/变速箱/过户次数/上架时间/特殊标记[0,0,0,0]/综合车况/外观检测")
count = 0
for j in new_table:
    count += 1
    print(j)  
    if count == 10:
        break

汽车品牌/二手车价格/新车购入价格/行驶里程/使用时间/车牌所在地/外迁标准[0,0]/变速箱/过户次数/上架时间/特殊标记[0,0,0,0]/综合车况/外观检测
[1.0, 1, 4.89, 7.94, 86, 1, 0, 0, 0, 1, 3, 0, 1, 0, 0, 3, 2]
[1.0, 1, 13.87, 10.71, 142, 1, 0, 1, 1, 1, 3, 1, 0, 0, 0, 3, 0]
[1.88, 1, 16.16, 9.12, 148, 1, 0, 1, 1, 4, 3, 0, 0, 0, 0, 3, 4]
[1.4, 0, 9.17, 10.07, 144, 0, 0, 1, 0, 0, 3, 1, 1, 1, 0, 2, 5]
[1.68, 0, 6.98, 14.24, 77, 0, 0, 1, 0, 1, 3, 1, 0, 0, 0, 3, 0]
[1.5, 1, 4.58, 11.36, 98, 1, 0, 1, 0, 0, 3, 1, 1, 1, 0, 2, 2]
[1.8, 0, 7.49, 10.01, 96, 0, 0, 1, 0, 2, 3, 1, 0, 0, 0, 2, 2]
[1.8, 1, 8.53, 10.33, 112, 1, 0, 1, 0, 0, 3, 1, 0, 1, 0, 3, 2]
[1.1, 1, 4.89, 3.33, 78, 2, 0, 1, 0, 0, 15, 1, 1, 1, 0, 2, 1]
[2.0, 0, 9.68, 3.85, 129, 1, 0, 1, 1, 0, 3, 0, 0, 1, 0, 3, 1]


In [6]:
#写入csv文件
 
name=['二手车价格','品牌', '新车购入价格','行驶里程','使用时间','车牌所在地','外迁标准1','外迁标准2','变速箱','过户次数','上架时间','超值','急售','0过户','准新车','综合车况','外观检测']
test=pd.DataFrame(columns=name,data=new_table)
test.to_csv('data/car_detail_new.csv', encoding='gbk')

print("写入完成")

写入完成
